IMPORTS

In [ ]:
import os
import sys
import pandas as pd
import numpy as np

In [ ]:
from prog import scheduler as sch

In [ ]:
# save current working directory as homey
# homey = os.path.abspath(os.path.dirname(__file__))
homey = os.getcwd() # works in jupyter notebook

In [ ]:
# set directory paths
dataPath = os.path.join(homey, 'data')
progPath = os.path.join(homey, 'prog')
simPath = os.path.join(homey, 'FB_Sim')
forcPath = os.path.join(simPath, 'ForecastRedoux')
# this should probably be moved out of the simulator and into the main sql area, where that ends up
sqlPath = os.path.join(forcPath, 'SQL')

In [ ]:
# set paths to excel files
forecastFilename = os.path.join(dataPath, 'RegularForecast.xlsx')
mfgCentersFilename = os.path.join(dataPath, 'MfgCenters.xlsx')
moFilename = os.path.join(dataPath, 'MOs.xlsx')
laborAvailFilename = os.path.join(dataPath, 'LaborAvailablePerDay.xlsx')
leadFilename = os.path.join(dataPath, 'LeadTimes.xlsx')

QUERIES

In [ ]:
sys.path.insert(0, forcPath)

In [ ]:
import ForecastMain as fm
import ForecastAPI as fa

In [ ]:
# pull the usual FB_Sim queries
# fa.run_queries(queryPath=sqlPath, dataPath=dataPath)

GET DATA

In [ ]:
# save mfgCenters as df, includes MFG Center assignments and Setup/labor time estimates
mfgCenters = pd.read_excel(mfgCentersFilename, header=0)

In [ ]:
# save current Manufacture Orders
modf = pd.read_excel(moFilename, header=0)

In [ ]:
# save lead time estimates
leadTimes = pd.read_excel(leadFilename, header=0)

In [ ]:
# sort by line priority before choosing which lead time to go with
leadTimes.sort_values(['PART','DefaultVendor','LastDate'], ascending=[True,False,False], inplace=True)
leadTimes.drop_duplicates('PART', keep='first', inplace=True)

In [ ]:
# use the Real Lead Time first and if it's not there take the vendor lead.
leadTimes['LeadTimes'] = np.nan
x=0
while x < len(leadTimes):
    if leadTimes['RealLeadTime'].iat[x] > 0:
        leadTimes['LeadTimes'].iat[x] = leadTimes['RealLeadTime'].iat[x]
    elif leadTimes['VendorLeadTime'].iat[x] > 0:
        leadTimes['LeadTimes'].iat[x] = leadTimes['VendorLeadTime'].iat[x]
    x+=1

In [ ]:
leadTimes = leadTimes[['PART','Make/Buy','AvgCost','LeadTimes']].copy()

In [ ]:
### this is a bandaid, I think there will be problems with NAN values later.  Need to figure out eventually.
leadTimes.fillna(10, inplace=True)

In [ ]:
# this is a placeholder for a calculation of start to finish time for a build.
# just using it for earliest schedule date right now.
orderRunTime = 7

MAKE DATE LIST

In [ ]:
# creating a common timestamp, if not added they can all generate their own seconds off from each other
todayTimestamp = pd.Timestamp.today()

In [ ]:
# make a date list with labor availability
dateList = sch.create_date_list(todayTimestamp=todayTimestamp, dailyLabor=11)

In [ ]:
## labor type testing
dateListProLine = sch.create_date_list(todayTimestamp=todayTimestamp, dailyLabor=40)
dateListRacking = sch.create_date_list(todayTimestamp=todayTimestamp, dailyLabor=12)
dateListPCB = sch.create_date_list(todayTimestamp=todayTimestamp, dailyLabor=24)
dateListLabels = sch.create_date_list(todayTimestamp=todayTimestamp, dailyLabor=7)
dateListKitting = sch.create_date_list(todayTimestamp=todayTimestamp, dailyLabor=12)
dateListShipping = sch.create_date_list(todayTimestamp=todayTimestamp, dailyLabor=6)
dateListCableAssy = sch.create_date_list(todayTimestamp=todayTimestamp, dailyLabor=6)

CREATE IDEAL SCHEDULE

In [ ]:
# prep mo list with mfg centers and labor estimates
preppedMOdf = sch.pre_schedule_prep(modf=modf.copy(), mfgCenters=mfgCenters.copy())

In [ ]:
# add an empty datetime column
#   - this allows the use of the same scheduling formula throughout the script
preppedMOdf['EarliestScheduleDate'] = np.nan
preppedMOdf['EarliestScheduleDate'] = pd.to_datetime(preppedMOdf['EarliestScheduleDate'])

# run the auto schedule to get an ideal schedule by priority
moLinesLabor = sch.sched_with_date_limits(orderPriority=preppedMOdf.copy(), dateList=dateList.copy())

In [ ]:
## labor type testing
moLinesLaborProLine = sch.run_auto_schedule(moLinesLabor=preppedMOdf[preppedMOdf['Mfg Center'] == 'Pro Line'].copy(), dateList=dateListProLine.copy())
moLinesLaborRacking = sch.run_auto_schedule(moLinesLabor=preppedMOdf[preppedMOdf['Mfg Center'] == 'Racking'].copy(), dateList=dateListRacking.copy())
moLinesLaborPCB = sch.run_auto_schedule(moLinesLabor=preppedMOdf[preppedMOdf['Mfg Center'] == 'PCB'].copy(), dateList=dateListPCB.copy())
moLinesLaborLabels = sch.run_auto_schedule(moLinesLabor=preppedMOdf[preppedMOdf['Mfg Center'] == 'Labels'].copy(), dateList=dateListLabels.copy())
moLinesLaborKitting = sch.run_auto_schedule(moLinesLabor=preppedMOdf[preppedMOdf['Mfg Center'] == 'Kitting'].copy(), dateList=dateListKitting.copy())
moLinesLaborShipping = sch.run_auto_schedule(moLinesLabor=preppedMOdf[preppedMOdf['Mfg Center'] == 'Shipping'].copy(), dateList=dateListShipping.copy())
moLinesLaborCableAssy = sch.run_auto_schedule(moLinesLabor=preppedMOdf[preppedMOdf['Mfg Center'] == 'Cable Assembly'].copy(), dateList=dateListCableAssy.copy())

In [ ]:
## labor type testing
moLinesLaborProLine.drop_duplicates('ORDER', keep='last', inplace=True)
moLinesLaborRacking.drop_duplicates('ORDER', keep='last', inplace=True)
moLinesLaborPCB.drop_duplicates('ORDER', keep='last', inplace=True)
moLinesLaborLabels.drop_duplicates('ORDER', keep='last', inplace=True)
moLinesLaborKitting.drop_duplicates('ORDER', keep='last', inplace=True)
moLinesLaborShipping.drop_duplicates('ORDER', keep='last', inplace=True)
moLinesLaborCableAssy.drop_duplicates('ORDER', keep='last', inplace=True)

In [ ]:
## labor type testing
idealSchedule = pd.concat([moLinesLaborProLine,
		   				   moLinesLaborRacking,
		   				   moLinesLaborPCB,
		   				   moLinesLaborLabels,
		   				   moLinesLaborKitting,
		   				   moLinesLaborShipping,
		   				   moLinesLaborCableAssy])

# use the last scheduled FG in an order to save an ideal schedule
idealSchedule = moLinesLabor.drop_duplicates('ORDER', keep='last')

RUN THE SIM

In [ ]:
# replace the schedule dates on the MO order lines with the new dates for those orders
newMOdf = pd.merge(modf.copy(), idealSchedule[['ORDER', 'NewDate']].copy(), how='left', on='ORDER')
newMOdf['DATESCHEDULED'] = newMOdf['NewDate'].copy()
newMOdf.drop(labels='NewDate', axis=1, inplace=True)

In [ ]:
# run the new MO schedule through the FB_Sim to find phantom orders
orderTimeline = fm.run_normal_forecast_tiers_v3(dataPath=dataPath, includeSO=False, subMO=newMOdf.copy())

GET SCHEDULE LIMITS

In [ ]:
orderLeads = sch.get_earliest_leads(orderTimeline=orderTimeline.copy(),
                                    leadTimes=leadTimes.copy(),
                                    dateList=dateList.copy(),
                                    orderRunTime=orderRunTime)

ANALYZE SCHEDULE

# analyze and adjust the schedule
newMOdf = sch.analyze_schedule(newMOdf=newMOdf.copy(),
                               orderLeads=orderLeads.copy(),
                               modf=modf.copy(),
                               mfgCenters=mfgCenters.copy(),
                               dateList=dateList.copy(),
                               orderRunTime=orderRunTime,
                               leadTimes=leadTimes.copy())

In [ ]:
# Trying something with multiple labor types, not totally into this though.  requires hard coding each type.
def analyze_schedule_labor_types(newMOdf, orderLeads, modf, mfgCenters, dateList, orderRunTime, leadTimes, dateListProLine, dateListRacking, dateListPCB, dateListLabels, dateListKitting, dateListShipping, dateListCableAssy):
    print('in analyze_schedule')
    tempMOdf = newMOdf.sort_values(by=['ORDER','DATESCHEDULED'], ascending=[True, True]).copy()
    tempMOdf.drop_duplicates('ORDER', keep='first', inplace=True)
    checkSched = pd.merge(tempMOdf[['ORDER','DATESCHEDULED']].copy(),
                          orderLeads[['ORDER','EarliestScheduleDate']].copy(),
                          how='left', on='ORDER')
    checkSched['TimeDiff'] = np.nan
    # need to convert 'DATESCHEDULED' column to datetime or it will register as a float and error out in comparison
    checkSched['DATESCHEDULED'] = pd.to_datetime(checkSched['DATESCHEDULED'].copy())
    for each in range(0, len(checkSched)):
        if checkSched['DATESCHEDULED'].iat[each] < checkSched['EarliestScheduleDate'].iat[each]:
            checkSched['TimeDiff'] = 'here'
            print(checkSched['ORDER'].iat[each])
    if len(checkSched.dropna()) != 0:
        newMOdf = schedule_loop_labor_types(modf=modf.copy(),
                                  orderLeads=orderLeads.copy(),
                                  mfgCenters=mfgCenters.copy(),
                                  dateList=dateList.copy(),
                                  orderRunTime=orderRunTime,
                                  leadTimes=leadTimes.copy(),
                                  dateListProLine=dateListProLine.copy(),
                                  dateListRacking=dateListRacking.copy(),
                                  dateListPCB=dateListPCB.copy(),
                                  dateListLabels=dateListLabels.copy(),
                                  dateListKitting=dateListKitting.copy(),
                                  dateListShipping=dateListShipping.copy(),
                                  dateListCableAssy=dateListCableAssy.copy())
    else:
        print('no schedule issues found')
        return(newMOdf.copy())
    return(newMOdf.copy())

In [ ]:
# adjusts schedule dates and runs a sim.  Uses analyze_schedule() to check its result.
def schedule_loop_labor_types(modf, orderLeads, mfgCenters, dateList, orderRunTime, leadTimes, dateListProLine, dateListRacking, dateListPCB, dateListLabels, dateListKitting, dateListShipping, dateListCableAssy):
    print('in schedule_loop')
    ### CREATE NEW SCHEDULE ###

    # save a new copy of the modf with longest leads added
    leadMOdf = pd.merge(modf.copy(), orderLeads[['ORDER','EarliestScheduleDate']].copy(), how='left', on='ORDER')
    moLinesLabor = sch.pre_schedule_prep(modf=leadMOdf, mfgCenters=mfgCenters.copy())

    outputScheduleProLine = sch.sched_with_date_limits(orderPriority=moLinesLabor[moLinesLabor['Mfg Center'] == 'Pro Line'].copy(), dateList=dateListProLine.copy())
    outputScheduleRacking = sch.sched_with_date_limits(orderPriority=moLinesLabor[moLinesLabor['Mfg Center'] == 'Racking'].copy(), dateList=dateListRacking.copy())
    outputSchedulePCB = sch.sched_with_date_limits(orderPriority=moLinesLabor[moLinesLabor['Mfg Center'] == 'PCB'].copy(), dateList=dateListPCB.copy())
    outputScheduleLabels = sch.sched_with_date_limits(orderPriority=moLinesLabor[moLinesLabor['Mfg Center'] == 'Labels'].copy(), dateList=dateListLabels.copy())
    outputScheduleKitting = sch.sched_with_date_limits(orderPriority=moLinesLabor[moLinesLabor['Mfg Center'] == 'Kitting'].copy(), dateList=dateListKitting.copy())
    outputScheduleShipping = sch.sched_with_date_limits(orderPriority=moLinesLabor[moLinesLabor['Mfg Center'] == 'Shipping'].copy(), dateList=dateListShipping.copy())
    outputScheduleCableAssy = sch.sched_with_date_limits(orderPriority=moLinesLabor[moLinesLabor['Mfg Center'] == 'Cable Assembly'].copy(), dateList=dateListCableAssy.copy())

    outputScheduleProLine.drop_duplicates('ORDER', keep='last', inplace=True)
    outputScheduleRacking.drop_duplicates('ORDER', keep='last', inplace=True)
    outputSchedulePCB.drop_duplicates('ORDER', keep='last', inplace=True)
    outputScheduleLabels.drop_duplicates('ORDER', keep='last', inplace=True)
    outputScheduleKitting.drop_duplicates('ORDER', keep='last', inplace=True)
    outputScheduleShipping.drop_duplicates('ORDER', keep='last', inplace=True)
    outputScheduleCableAssy.drop_duplicates('ORDER', keep='last', inplace=True)

    newSchedule = pd.concat([outputScheduleProLine,
                             outputScheduleRacking,
                             outputSchedulePCB,
                             outputScheduleLabels,
                             outputScheduleKitting,
                             outputScheduleShipping,
                             outputScheduleCableAssy])
    print('made newSchedule')
    # outputSchedule = sched_with_date_limits(orderPriority=moLinesLabor.copy(),
    #                                             dateList=dateList.copy())
    # use the last scheduled FG in an order to save a new schedule
    # newSchedule = outputSchedule.drop_duplicates('ORDER', keep='last').copy()

    ### RUN THE SIM ###

    # replace the schedule dates on the MO order lines with the new dates for those orders
    newMOdf = pd.merge(modf.copy(), newSchedule[['ORDER', 'NewDate']].copy(), how='left', on='ORDER')
    newMOdf['DATESCHEDULED'] = newMOdf['NewDate'].copy()
    newMOdf.drop(labels='NewDate', axis=1, inplace=True)
    
    print('going into FB_Sim')
    # run the new MO schedule through the FB_Sim to find phantom orders
    orderTimeline = fm.run_normal_forecast_tiers_v3(dataPath=dataPath, includeSO=False, subMO=newMOdf.copy())
    print('out of FB_Sim')

    ### GET SCHEDULE LIMITS ###

    # get a fresh list of earliest leads per order from the recent sim run
    freshLeads = sch.get_earliest_leads(orderTimeline=orderTimeline.copy(),
                                        leadTimes=leadTimes.copy(),
                                        dateList=dateList.copy(),
                                        orderRunTime=orderRunTime)
    # combine it with any previous lists to get the last schedule date per order
    orderLeads = sch.combine_order_leads(oldLeads=orderLeads.copy(), newLeads=freshLeads.copy())
    print('try to analyze again')

    newMOdf = analyze_schedule_labor_types(newMOdf=newMOdf.copy(),
                                            orderLeads=orderLeads.copy(),
                                            modf=modf.copy(),
                                            mfgCenters=mfgCenters.copy(),
                                            dateList=dateList.copy(),
                                            orderRunTime=orderRunTime,
                                            leadTimes=leadTimes.copy(),
                                            dateListProLine=dateListProLine.copy(),
                                            dateListRacking=dateListRacking.copy(),
                                            dateListPCB=dateListPCB.copy(),
                                            dateListLabels=dateListLabels.copy(),
                                            dateListKitting=dateListKitting.copy(),
                                            dateListShipping=dateListShipping.copy(),
                                            dateListCableAssy=dateListCableAssy.copy())
    return(newMOdf.copy())

In [ ]:
bork = analyze_schedule_labor_types(newMOdf=newMOdf.copy(),
                     orderLeads=orderLeads.copy(),
                     modf=modf.copy(),
                     mfgCenters=mfgCenters.copy(),
                     dateList=dateList.copy(),
                     orderRunTime=orderRunTime,
                     leadTimes=leadTimes.copy(),
                     dateListProLine=dateListProLine.copy(),
                     dateListRacking=dateListRacking.copy(),
                     dateListPCB=dateListPCB.copy(),
                     dateListLabels=dateListLabels.copy(),
                     dateListKitting=dateListKitting.copy(),
                     dateListShipping=dateListShipping.copy(),
                     dateListCableAssy=dateListCableAssy.copy())

In [ ]:
bork.to_clipboard()

In [ ]:
print('in analyze_schedule')
tempMOdf = newMOdf.sort_values(by=['ORDER','DATESCHEDULED'], ascending=[True, True]).copy()
tempMOdf.drop_duplicates('ORDER', keep='first', inplace=True)
checkSched = pd.merge(tempMOdf[['ORDER','DATESCHEDULED']].copy(),
					  orderLeads[['ORDER','EarliestScheduleDate']].copy(),
					  how='left', on='ORDER')
checkSched['TimeDiff'] = np.nan
# need to convert 'DATESCHEDULED' column to datetime or it will register as a float and error out in comparison
checkSched['DATESCHEDULED'] = pd.to_datetime(checkSched['DATESCHEDULED'].copy())
for each in range(0, len(checkSched)):
    if checkSched['DATESCHEDULED'].iat[each] < checkSched['EarliestScheduleDate'].iat[each]:
        checkSched['TimeDiff'] = 'here'
        print(checkSched['ORDER'].iat[each])

In [ ]:
leadMOdf = pd.merge(modf.copy(), orderLeads[['ORDER','EarliestScheduleDate']].copy(), how='left', on='ORDER')
moLinesLabor = sch.pre_schedule_prep(modf=leadMOdf, mfgCenters=mfgCenters.copy())

In [ ]:
outputScheduleProLine = sch.sched_with_date_limits(orderPriority=moLinesLabor[moLinesLabor['Mfg Center'] == 'Pro Line'].copy(), dateList=dateListProLine.copy())

In [ ]:
orderPriority = moLinesLabor[moLinesLabor['Mfg Center'] == 'Pro Line'].copy()

In [ ]:
outputSchedule = pd.DataFrame(columns=['ORDER','LaborRequired','EarliestScheduleDate','NewDate'])
unusedLabor = 0
lH = 0

In [ ]:
lH = 9

In [ ]:
totalLabor = sch.labor_total(orderLabor=orderPriority['LaborRequired'].iat[lH],
                             usedLabor=outputSchedule['LaborRequired'].sum(),
                             extraLabor=unusedLabor)
tempDate = dateListProLine[dateListProLine['AvailableLabor'] >= totalLabor].head(1)
schedDate = tempDate['StartDate'].iat[0]

In [ ]:
lH+=1

In [ ]:
orderPriority.sort_values('EarliestScheduleDate', inplace=True)

In [ ]:
lH = 0

In [ ]:
schedDate = orderPriority['EarliestScheduleDate'].iat[0]

In [ ]:
schedDate

In [ ]:
outputSchedule = outputSchedule.append({'ORDER':orderPriority['ORDER'].iat[lH],
                                                    'LaborRequired':orderPriority['LaborRequired'].iat[lH],
                                                    'EarliestScheduleDate':orderPriority['EarliestScheduleDate'].iat[lH],
                                                    'NewDate':schedDate},
                                                    ignore_index=True)

In [ ]:
orderPriority.drop(orderPriority.index[lH], inplace=True)

In [ ]:
orderPriority.sort_values('DATESCHEDULED', inplace=True)

In [ ]:
tempDate

In [ ]:
tempDate = dateListProLine[dateListProLine['StartDate'] == schedDate]

In [ ]:
schedDate

In [ ]:
dateListProLine

In [ ]:
dateList

In [ ]:
while lH < len(orderPriority):
    print(lH)
    # collect labor needed and get relevant schedule date
    totalLabor = sch.labor_total(orderLabor=orderPriority['LaborRequired'].iat[lH],
                             usedLabor=outputSchedule['LaborRequired'].sum(),
                             extraLabor=unusedLabor)
    tempDate = dateListProLine[dateListProLine['AvailableLabor'] >= totalLabor].head(1)
    schedDate = tempDate['StartDate'].iat[0]
    if schedDate < orderPriority['EarliestScheduleDate'].iat[lH]:
        # if the schedule date is before the line can schedule then move on
        lH+=1
        if lH >= len(orderPriority):
            # if there are no more priorities then there can only be orders held by their earliest schedule dates
            # temporarily sort by earliest allowed date to find the next order to schedule
            orderPriority.sort_values('EarliestScheduleDate', inplace=True)
            lH = 0
            schedDate = orderPriority['EarliestScheduleDate'].iat[0]
            outputSchedule = outputSchedule.append({'ORDER':orderPriority['ORDER'].iat[lH],
                                                    'LaborRequired':orderPriority['LaborRequired'].iat[lH],
                                                    'EarliestScheduleDate':orderPriority['EarliestScheduleDate'].iat[lH],
                                                    'NewDate':schedDate},
                                                    ignore_index=True)
            orderPriority.drop(orderPriority.index[lH], inplace=True)
            orderPriority.sort_values('DATESCHEDULED', inplace=True)
            # now that it's set, we need to add value to unused labor (any labor skipped due to material timing)
            # otherwise overlapping earliest schedule dates could overlap and double up on available labor
            tempDate = dateListProLine[dateListProLine['StartDate'] == schedDate]
            availLabor = tempDate['AvailableLabor'].iat[0]
            usedLabor = outputSchedule['LaborRequired'].sum() + unusedLabor
            laborGap = availLabor - usedLabor
            unusedLabor = laborGap + unusedLabor
    else:
        print('schedule')
        # otherwise add a schedule line to the output and delete the order from the labor list
        outputSchedule = outputSchedule.append({'ORDER':orderPriority['ORDER'].iat[lH],
                                                'LaborRequired':orderPriority['LaborRequired'].iat[lH],
                                                'EarliestScheduleDate':orderPriority['EarliestScheduleDate'].iat[lH],
                                                'NewDate':schedDate},
                                                ignore_index=True)
        orderPriority.drop(orderPriority.index[lH], inplace=True)
        # set the iterator back to 0 to start back at the top of the remaining priority list
        lH = 0

In [ ]:
outputSchedule

In [ ]:
orderPriority

In [ ]:
orderPriority.to_clipboard()